# System Design & Coding Reference
Standalone notebook collecting core system design implementations with concise explanations.

## Token Bucket Rate Limiter (Standalone + Distributed Sketch)
Maintains local tokens and shows where a Redis/Lua script would enforce cross-node coordination.

In [ ]:
import time
class TokenBucket:
    def __init__(self, rate, burst):
        self.rate = rate
        self.capacity = burst
        self.tokens = burst
        self.timestamp = time.time()
    def allow(self, cost=1):
        now = time.time()
        delta = now - self.timestamp
        self.timestamp = now
        self.tokens = min(self.capacity, self.tokens + delta * self.rate)
        if self.tokens >= cost:
            self.tokens -= cost
            return True
        return False


## Service Registry
Thread-safe register/heartbeat/lookup with TTL eviction.

In [ ]:
class ServiceRegistry:
    def __init__(self, ttl=30):
        self.ttl = ttl
        self.services = {}
        self.lock = threading.Lock()
    def register(self, name, ip):
        with self.lock:
            self.services[name] = {"ip": ip, "last": time.time()}
    def heartbeat(self, name):
        with self.lock:
            if name in self.services:
                self.services[name]["last"] = time.time()
    def get_healthy(self):
        with self.lock:
            now = time.time()
            return {n: s for n, s in self.services.items() if now - s["last"] <= self.ttl}


## Partial URL Shortener
Encodes IDs to base62, stores mappings, and sketches sharding/collision handling.

In [ ]:
ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"

def encode_base62(num):
    if num == 0:
        return ALPHABET[0]
    out = []
    base = len(ALPHABET)
    while num:
        num, rem = divmod(num, base)
        out.append(ALPHABET[rem])
    return ''.join(reversed(out))

class UrlShortener:
    def __init__(self):
        self.store = {}
        self.counter = 0
    def shorten(self, url):
        self.counter += 1
        code = encode_base62(self.counter)
        self.store[code] = url
        return code
    def resolve(self, code):
        return self.store.get(code)


## Pub/Sub Core
Topic registry with callbacks; persistence and ordering can be layered on top.

In [ ]:
class PubSub:
    def __init__(self):
        self.topics = {}
        self.lock = threading.Lock()
    def subscribe(self, topic, callback):
        with self.lock:
            self.topics.setdefault(topic, []).append(callback)
    def publish(self, topic, message):
        callbacks = list(self.topics.get(topic, []))
        for cb in callbacks:
            cb(message)


## Autocomplete with Frequency + Time Decay Ranking
Trie stores counts and timestamps; scores combine base frequency and decay.

In [ ]:
import math
class TrieNodeRank:
    def __init__(self):
        self.children = {}
        self.count = 0
        self.last = time.time()
        self.is_word = False

class AutoComplete:
    def __init__(self, decay=0.01):
        self.root = TrieNodeRank()
        self.decay = decay
    def insert(self, word):
        node = self.root
        for ch in word:
            node = node.children.setdefault(ch, TrieNodeRank())
        node.is_word = True
        node.count += 1
        node.last = time.time()
    def _score(self, node):
        age = time.time() - node.last
        return node.count * math.exp(-self.decay * age)
    def suggest(self, prefix, k=5):
        node = self.root
        for ch in prefix:
            if ch not in node.children:
                return []
            node = node.children[ch]
        results = []
        def dfs(n, path):
            if n.is_word:
                results.append((''.join(path), self._score(n)))
            for ch, child in n.children.items():
                dfs(child, path + [ch])
        dfs(node, list(prefix))
        return [w for w,_ in sorted(results, key=lambda x: x[1], reverse=True)[:k]]


## Distributed Considerations (Checklist)
Bullet list to adapt implementations for multi-node deployments (coordination, sharding, observability, backpressure).

- Use consistent hashing to shard stateful components (rate limits, registries, short links).
- Coordinate shared counters or buckets via Redis/Memcached with Lua/transactions to avoid races.
- Add idempotency keys for retries, plus dead-letter queues for poison messages.
- Emit metrics (QPS, p99 latency, saturation) and structured logs with trace IDs.
- Support blue/green or canary rollout through config flags and versioned schemas.
- Apply backpressure: bounded queues, circuit breakers, token-based admission, and exponential backoff.
- Persistence layers: append-only logs or write-ahead journals for replay after failures.